In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [23]:
## load the dataset
df = pd.read_csv("Churn_Modelling.csv")

In [24]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [27]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Length: 10000, dtype: bool

In [28]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [29]:
##dropping the columns which are irrelevant
df.drop(["RowNumber","CustomerId","Surname"],axis=1,inplace=True)


In [30]:
## encoding the categorical columns
label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])


In [31]:
## encoding the Geography column by using the one hot encoder
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(df[["Geography"]])

In [32]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [33]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [34]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

In [35]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [36]:
## combining the encoder columns with the real data
df = pd.concat([df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [37]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [38]:
## save the pickle file of encoder
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo,file)

In [39]:
## divide the data into independent and dependent columns
X = df.drop('Exited',axis=1)
y = df["Exited"]



In [40]:
## train test splitting 
##splitting the data into train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## Scalar these
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [41]:
## saving the scalar pickle file
with open("scalar.pkl","wb")as file:
    pickle.dump(scalar,file)

ANN Implementation

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

In [44]:
models = Sequential([
    Dense(64,activation="relu",input_shape = (X_train.shape[1],)),
    Dense(32,activation="relu"),
    Dense(1,activation="sigmoid")
])

In [57]:
X_train.shape

(8000, 12)

In [47]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [54]:
models.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

In [68]:
## set up the tensorboard 
from datetime import datetime
log_dir = "logs/fit"+ datetime.now().strftime("%Y-%m-%d %H-%M-%S")
tensor_board = TensorBoard(log_dir)


In [50]:
## set the EarlyStopping
early_stopping = EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)


In [52]:
X_train.shape

(8000, 12)

In [53]:
y_train.shape

(8000,)

In [59]:
history = models.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks = [tensor_board,early_stopping]
)

Epoch 1/100


250/250 [==============================] - 3s 3ms/step - loss: 0.3974 - accuracy: 0.8335 - val_loss: 0.3590 - val_accuracy: 0.8500
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3529 - accuracy: 0.8539 - val_loss: 0.3457 - val_accuracy: 0.8610
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3452 - accuracy: 0.8580 - val_loss: 0.3441 - val_accuracy: 0.8585
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3417 - accuracy: 0.8606 - val_loss: 0.3508 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3412 - accuracy: 0.8599 - val_loss: 0.3411 - val_accuracy: 0.8620
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3388 - accuracy: 0.8614 - val_loss: 0.3429 - val_accuracy: 0.8575
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3355 - accuracy: 0.8618 - val_loss: 0.3550 - val_accuracy: 0.84

In [60]:
models.save("models.h5")

c:\Users\piyus\OneDrive\Desktop\Churn Modeling\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [63]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
